In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import tensorflow as tf

In [44]:
df = pd.read_csv('./data/AirPassengers.csv')
df.head()

,Month,#Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [45]:
df.tail()

,Month,#Passengers
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390
143,1960-12,432


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Month        144 non-null    object
 1   #Passengers  144 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ KB


# Q.과거 5개월의 데이터를 학습해서 다음 년도의 탑승객 수를 예측하는 모델을 만들고 1960-07 ~ 1960-11의 5개월 데이터를 기반으로 1960-12월의 탑승객 수를 예측하시오

In [92]:
# month 카테고리 
df['mon'] = df['Month'].apply(lambda x:x.split('-')[1])
df['mon']

0      01
1      02
2      03
3      04
4      05
       ..
139    08
140    09
141    10
142    11
143    12
Name: mon, Length: 144, dtype: object

In [93]:
df

,Month,#Passengers,mon
0,1949-01,112,01
1,1949-02,118,02
2,1949-03,132,03
3,1949-04,129,04
4,1949-05,121,05
...,...,...,...
139,1960-08,606,08
140,1960-09,508,09
141,1960-10,461,10
142,1960-11,390,11


In [94]:
df.groupby('mon')['#Passengers'].mean()

mon
01    241.750000
02    235.000000
03    270.166667
04    267.083333
05    271.833333
06    311.666667
07    351.333333
08    351.083333
09    302.416667
10    266.583333
11    232.833333
12    261.833333
Name: #Passengers, dtype: float64

In [95]:
# 마지막 1960-12: test데이터

In [96]:
# month 원핫 벡터로 만들기
# 지금까지 x는 get_dummies y값만 to_categorical

from tensorflow.keras.utils import to_categorical
df_mon = to_categorical(df.mon)

In [97]:
df_mon

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [98]:
df_month = pd.DataFrame(df_mon)
df_month

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [99]:
df_con = pd.concat([df,df_month],axis=1)
df_con

,Month,#Passengers,mon,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1949-01,112,01,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1949-02,118,02,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1949-03,132,03,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1949-04,129,04,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1949-05,121,05,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,1960-08,606,08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
140,1960-09,508,09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
141,1960-10,461,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
142,1960-11,390,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [100]:
np.unique(df_con[0])

array([0.], dtype=float32)

In [101]:
# 완성된 특성
df_con.drop(columns=['mon','Month',0],inplace=True)
df_con

,#Passengers,1,2,3,4,5,6,7,8,9,10,11,12
0,112,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,118,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,132,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,129,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,121,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
140,508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
141,461,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
142,390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [102]:
# test 1960-12 제외
df_train = df_con.iloc[:-1,:]
df_test = df_con.iloc[-1,:]

In [105]:
len(df_con)

144

In [108]:
# 5일뒤의 승객 : y
window_size = 5
X=[]
Y=[]

for i in range(len(df_con)-window_size):
    X.append([df_con.iloc[i+j] for j in range(window_size)])
    Y.append(df_con.iloc[window_size+i])

In [109]:
X = np.array(X)
Y = np.array(Y)

In [110]:
X.shape

(139, 5, 13)

In [123]:
model = Sequential()
model.add(LSTM(15 ,input_shape = (5,13),activation= 'relu',return_sequences=True))
model.add(LSTM(15 ,activation= 'relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 5, 15)             1740      
                                                                 
 lstm_6 (LSTM)               (None, 15)                1860      
                                                                 
 dropout (Dropout)           (None, 15)                0         
                                                                 
 dense_3 (Dense)             (None, 128)               2048      
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 1)                

In [124]:
model.compile(loss='mse', metrics=['mse'], optimizer ='adam')

In [125]:
hist = model.fit(X,Y, epochs=100,batch_size=10)

Epoch 1/100
14/14 [==============================] - 2s 40ms/step - loss: 6976.3989 - mse: 6976.3989
Epoch 2/100
14/14 [==============================] - 0s 31ms/step - loss: 6936.6143 - mse: 6936.6143
Epoch 3/100
14/14 [==============================] - 0s 32ms/step - loss: 6917.7471 - mse: 6917.7471
Epoch 4/100
14/14 [==============================] - 0s 30ms/step - loss: 6853.4282 - mse: 6853.4282
Epoch 5/100
14/14 [==============================] - 0s 31ms/step - loss: 6848.2441 - mse: 6848.2441
Epoch 6/100
14/14 [==============================] - 0s 28ms/step - loss: 6843.9019 - mse: 6843.9019
Epoch 7/100
14/14 [==============================] - 1s 36ms/step - loss: 6847.9731 - mse: 6847.9731
Epoch 8/100
14/14 [==============================] - 0s 32ms/step - loss: 6854.5684 - mse: 6854.5684
Epoch 9/100
14/14 [==============================] - 0s 30ms/step - loss: 6830.0190 - mse: 6830.0190
Epoch 10/100
14/14 [==============================] - 0s 31ms/step - loss: 6846.9893 - mse:

14/14 [==============================] - 0s 30ms/step - loss: 6814.1235 - mse: 6814.1235
Epoch 82/100
14/14 [==============================] - 0s 28ms/step - loss: 6810.1562 - mse: 6810.1562
Epoch 83/100
14/14 [==============================] - 0s 28ms/step - loss: 6808.8901 - mse: 6808.8901
Epoch 84/100
14/14 [==============================] - 0s 28ms/step - loss: 6812.3491 - mse: 6812.3486
Epoch 85/100
14/14 [==============================] - 0s 28ms/step - loss: 6807.5942 - mse: 6807.5942
Epoch 86/100
14/14 [==============================] - 0s 30ms/step - loss: 6811.2788 - mse: 6811.2788
Epoch 87/100
14/14 [==============================] - 0s 35ms/step - loss: 6810.4634 - mse: 6810.4634
Epoch 88/100
14/14 [==============================] - 1s 37ms/step - loss: 6807.6309 - mse: 6807.6309
Epoch 89/100
14/14 [==============================] - 0s 31ms/step - loss: 6808.1431 - mse: 6808.1426
Epoch 90/100
14/14 [==============================] - 0s 31ms/step - loss: 6802.9980 - mse: 680

In [126]:
model.evaluate(X,Y)

5/5 [==============================] - 0s 8ms/step - loss: 6795.1582 - mse: 6795.1582


[6795.158203125, 6795.158203125]

In [127]:
x_train_pred

array([[128.9048  ],
       [145.54912 ],
       [156.25342 ],
       [152.34326 ],
       [142.08086 ],
       [118.282455],
       [115.52792 ],
       [125.653656],
       [133.16362 ],
       [135.51387 ],
       [138.9363  ],
       [139.55588 ],
       [132.03185 ],
       [157.81764 ],
       [175.44025 ],
       [166.76344 ],
       [160.69058 ],
       [126.086174],
       [129.98256 ],
       [146.93571 ],
       [166.24556 ],
       [154.77098 ],
       [173.40924 ],
       [168.30399 ],
       [180.17265 ],
       [190.84329 ],
       [205.0881  ],
       [202.46109 ],
       [193.79323 ],
       [151.38017 ],
       [153.3102  ],
       [169.46985 ],
       [189.864   ],
       [187.20361 ],
       [191.81212 ],
       [193.02626 ],
       [199.07887 ],
       [223.39201 ],
       [227.55525 ],
       [233.73929 ],
       [202.87439 ],
       [175.76399 ],
       [182.28062 ],
       [197.31406 ],
       [212.77171 ],
       [213.86418 ],
       [232.97679 ],
       [235.5

In [128]:
test_df = df_train[-5:].values.reshape(1, 5, 13)

In [129]:
model.predict(test_df)

array([[33.701206]], dtype=float32)

In [ ]:
model.predict(X)